<a href="https://colab.research.google.com/github/VickkiMars/AI-ML/blob/main/human_vs_ai_text_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od

In [ ]:
od.download("https://www.kaggle.com/datasets/shanegerami/ai-vs-human-text/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: victorthamartian
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/shanegerami/ai-vs-human-text


100%|██████████| 350M/350M [00:20<00:00, 18.2MB/s]


In [ ]:
import pandas as pd

In [29]:
data = pd.read_csv("/content/ai-vs-human-text/AI_Human.csv")

In [30]:
!pip install -q --upgrade keras-nlp
!pip install -q -upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.4/548.4 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 103.9 MB/s eta 0:00:00

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [31]:
import keras_nlp
import keras

keras.mixed_precision.set_global_policy("mixed_float16")

In [32]:
import tensorflow as tf

In [60]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('/content/ai-vs-human-text/AI_Human.csv')

train, test = train_test_split(data, test_size=0.2, random_state=42)

train.to_csv('ai_human_train.csv', index=False)
test.to_csv('ai_human_test.csv', index=False)

In [63]:
BATCH_SIZE = 16
ai_human_train = tf.data.experimental.CsvDataset(
    "/content/ai_human_train.csv",
    [tf.string, tf.float64],
    header=True,
).shuffle(buffer_size=1000).batch(BATCH_SIZE)

ai_human_test = tf.data.experimental.CsvDataset(
    "/content/ai_human_test.csv",
    [tf.string, tf.float64],
    header=True,
).shuffle(buffer_size=1000).batch(BATCH_SIZE)

In [65]:
print(ai_human_train.unbatch().batch(5).take(1).get_single_element()[0][0])

tf.Tensor(b"Many students have been either forced or pressured to do an extracurricular activity. They might be forced or pressured by a parent/guardian, teacher, or friend. However, that doesn't mean they should do the activity. The principal should not force students to participate in an extracurricular activity because, students might not have time to do an activity, might be forced to do an activity they don't like, or the activities might suffer.\n\nWhile some students have plenty of extra time to use after school, some may not. Students can usually RET their homework done either after school or during resource. Though, they don't always do it in a healthy way. Some have to stay up late to RET an assignment done either because, they forgot when it was due, waited till the last minute to do it, didn't realize how BIR it was, or have a field trip or special event they are going to. This causes the student to not RET a healthy amount of sleep, which causes them to have trouble paying

In [66]:
classifier = keras_nlp.models.BertClassifier.from_preset("bert_base_en_uncased", num_classes=2)

In [67]:
import numpy as np

In [68]:
h = np.asarray(classifier.predict(["I love you"]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step


In [69]:
layer = tf.keras.layers.Softmax()
layer(h).numpy()

array([[0.43362167, 0.56637836]], dtype=float32)

In [70]:
classifier.evaluate(ai_human_test)

6091/6091 ━━━━━━━━━━━━━━━━━━━━ 863s 140ms/step - loss: 0.7918 - sparse_categorical_accuracy: 0.3757


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


[0.7933933138847351, 0.3729514479637146]

## Intermediate

In [ ]:
classifier.fit(
    ai_human_train,
    validation_data=ai_human_test,
    epochs=1,
)

   9812/Unknown 4519s 447ms/step - loss: 0.0592 - sparse_categorical_accuracy: 0.9790